In [1]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
import re

import keras, tensorflow
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

import json
import random
import pickle

from nltk.stem import WordNetLemmatizer 
import string

In [2]:
# Importing Data
with open('GL Bot.json') as file:
    corp = json.load(file)

# Display corpus file
print(corp)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [3]:
# Initialization
corp1 = corp["intents"]
corp2 = [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with olympus', 'olypus is not a good tool', 'lot of problems with olympus', 'how to use olympus', 'teach me olympus'], 'responses': ['Link: Olympus wiki'], 'context_set': ''}, {'tag': 'SL', 'patterns': ['i am not able to understand svm', 'explain me how machine learning works', 'i am not able to understand naive bayes', 'i am not able to understand logistic regression', 'i am not able to understand ensemble techb=niques', 'i am not able to understand knn', 'i am not able to understand knn imputer', 'i am not able to understand cross validation', 'i am not able to understand boosting', 'i am not able to understand random forest', 'i am not able to understand ada boosting', 'i am not able to understand gradient boosting', 'machine learning', 'ML', 'SL', 'supervised learning', 'knn', 'logistic regression', 'regression', 'classification', 'naive bayes', 'nb', 'ensemble techniques', 'bagging', 'boosting', 'ada boosting', 'ada', 'gradient boosting', 'hyper parameters'], 'responses': ['Link: Machine Learning wiki '], 'context_set': ''}, {'tag': 'NN', 'patterns': ['what is deep learning', 'unable to understand deep learning', 'explain me how deep learning works', 'i am not able to understand deep learning', 'not able to understand neural nets', 'very diffult to understand neural nets', 'unable to understand neural nets', 'ann', 'artificial intelligence', 'artificial neural networks', 'weights', 'activation function', 'hidden layers', 'softmax', 'sigmoid', 'relu', 'otimizer', 'forward propagation', 'backward propagation', 'epochs', 'epoch', 'what is an epoch', 'adam', 'sgd'], 'responses': ['Link: Neural Nets wiki'], 'context_set': ''}, {'tag': 'Bot', 'patterns': ['what is your name', 'who are you', 'name please', 'when are your hours of opertions', 'what are your working hours', 'hours of operation', 'working hours', 'hours'], 'responses': ['I am your virtual learning assistant'], 'context_set': ''}, {'tag': 'Profane', 'patterns': ['what the hell', 'bloody stupid bot', 'do you think you are very smart', 'screw you', 'i hate you', 'you are stupid', 'jerk', 'you are a joke', 'useless piece of shit'], 'responses': ['Please use respectful words'], 'context_set': ''}, {'tag': 'Ticket', 'patterns': ['my problem is not solved', 'you did not help me', 'not a good solution', 'bad solution', 'not good solution', 'no help', 'wasted my time', 'useless bot', 'create a ticket'], 'responses': ['Tarnsferring the request to your PM'], 'context_set': ''}]

In [4]:
# Using extend to convert while Iterating
for elm2 in corp2:
  
    for elm1 in corp1:
        if elm2['tag'] == elm1['tag']:
            elm1['responses'].extend(elm2['responses'])
            break
    else:
        corp1.append(elm2)

In [5]:
print(corp1)

[{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?', 'Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye', 'I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no li

In [6]:
corp["intents"] = corp1

In [7]:
# Pre-Processing
train_sent = [] #tokens
train_labels = []    

label = []          
response = []       

for intent in corp ['intents']:
    for pattern in intent['patterns']:
        
        train_sent.append(pattern)
        train_labels.append(intent['tag'])
        
    response.append(intent['responses'])
        
    #Add missing tag if any
    if intent['tag'] not in label:
        label.append(intent['tag'])
        
num_classes = len(label)

In [8]:
print(train_sent)
print(train_labels) 
print(label) 
print(response) 

['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time', 'thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy', 'olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of problem with olympus', 'olypus is not a good tool', 'lot of problems with olympus', 'how to use olympus', 'teach me olympus', 'i am not able to understand svm', 'explain me how machine learning works', 'i am not able to understand naive bayes', 'i am n

In [9]:
# Converting target labels by using LabelEncoder()
encoder = LabelEncoder()
encoder.fit(train_labels)
train_labels = encoder.transform(train_labels)

In [10]:
# Vectorization using tokenizer class 

vocab_size = 100000
embedding_dim = 16
max_length = 20
oov_token = "<OOV>"

token = Tokenizer(num_words=vocab_size, oov_token=oov_token)
token.fit_on_texts(train_sent)

word_index = token.word_index

sequence = token.texts_to_sequences(train_sent)
padded_sequence = pad_sequences(sequence, truncating='post', maxlen=max_length)

pad_sequences makes all the text sequences into the same size.

In [11]:
# Training the Model
model = Sequential()

model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            1600000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 1,600,680
Trainable params: 1,600,680
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Fitting

epochs = 400
history = model.fit(padded_sequence, np.array(train_labels), epochs=epochs)

Epoch 1/400
4/4 [==============================] - 1s 24ms/step - loss: 2.0772 - accuracy: 0.1406
Epoch 2/400
4/4 [==============================] - 0s 19ms/step - loss: 2.0723 - accuracy: 0.1875
Epoch 3/400
4/4 [==============================] - 0s 24ms/step - loss: 2.0658 - accuracy: 0.1875
Epoch 4/400
4/4 [==============================] - 0s 24ms/step - loss: 2.0604 - accuracy: 0.1875
Epoch 5/400
4/4 [==============================] - 0s 20ms/step - loss: 2.0545 - accuracy: 0.1875
Epoch 6/400
4/4 [==============================] - 0s 26ms/step - loss: 2.0488 - accuracy: 0.1875
Epoch 7/400
4/4 [==============================] - 0s 24ms/step - loss: 2.0431 - accuracy: 0.1875
Epoch 8/400
4/4 [==============================] - 0s 22ms/step - loss: 2.0367 - accuracy: 0.1875
Epoch 9/400
4/4 [==============================] - 0s 20ms/step - loss: 2.0300 - accuracy: 0.1875
Epoch 10/400
4/4 [==============================] - 0s 26ms/step - loss: 2.0230 - accuracy: 0.1875
Epoch 11/400
4/4 [=

In [13]:
# Saving the MOdel
model.save("Chatbot")

# to save the fitted token
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: Chatbot/assets


In [15]:
# According to the video
def chat():
    # load trained model
    model = keras.models.load_model('Chatbot')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        token = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        encoder = pickle.load(enc)
    
    # parameters
    max_length = 20
    
    print ("Welcome to Great Learning.")
    print ("This is an automated chat. How may I help you?")
    print ("type 'quit' to end the session")
    
    # While loop to run the chatbot indefinetely
    while True:
        print("")
        print("User: ", end="")
        
        # Takes the user input and converts all characters to lowercase
        inp = input()
         
        # Defining the Chatbot's exit condition
        if inp.lower() == "quit":
            print("Thank you for visiting Great Learning.")
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_length))
        tag = encoder.inverse_transform([np.argmax(result)])


        count = 0
        matched_intent = None
        
        for i in corp['intents']:
             
            if i['tag'] == tag:
                print("Bot:", np.random.choice(i['responses']))

chat()

Welcome to Great Learning.
This is an automated chat. How may I help you?
type 'quit' to end the session

User: Hi
Bot: Hello! how can i help you ?

User: Who are you
Bot: I am your virtual learning assistant

User: Artificial Intelligence
Bot: Link: Neural Nets wiki

User: Support Vector Machine
Bot: Link: Machine Learning wiki 

User: SVM
Bot: Link: Machine Learning wiki 

User: Thank you
Bot: I hope I was able to assist you, Good Bye

User: quit
Thank you for visiting Great Learning.
